In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/splits/test-single.csv")

In [8]:
valid_imagery_set = set([
    url.split("/")[7]
    for url in df["image_fn"].values
])

In [9]:
valid_imagery_set

{'md_100cm_2017', 'va_100cm_2016'}

In [7]:
urls[0].split("/")[7]

'md_100cm_2017'

In [10]:
import os

In [11]:
os.path.basename(urls[0])

'm_3707501_ne_18_1_20170611.tif'